---
# for ipython setting
---

In [ ]:
import sys

def ipy_exit(*args):
    exit(keep_kernel=True)

sys.exit = ipy_exit

---
# Make data load gokart task
---

In [2]:
"""--------------------------"""
"""------ [ templete ] ------"""
"""--------------------------"""

import gokart
import luigi
import numpy as np


luigi.configuration.LuigiConfigParser.add_config_path('./conf/param.ini')
np.random.seed(57)


class SampleTask(gokart.TaskOnKart):
    task_namespace = 'sample'

In [3]:
"""----------------------------"""
"""------ [ sampleTask ] ------"""
"""----------------------------"""

from sklearn.datasets import load_iris
import pandas as pd


class LoadIrisData(SampleTask):
    '''load iris.'''
    def run(self):
        iris = load_iris() 
        df = pd.DataFrame(iris.data, columns=iris.feature_names)
        df['target'] = iris.target
        df['target_name'] = df.target.apply(lambda x: iris.target_names[x])
        self.dump(df)

In [4]:
# python main.py sample.LoadIrisData --local-scheduler --no-lock
gokart.run(['sample.LoadIrisData', '--local-scheduler', '--no-lock'])

[2019/09/27 16:45:24][luigi][INFO](setup_logging.py:80) logging configured via *.conf file
[2019/09/27 16:45:24][luigi-interface][INFO](worker.py:595) Informed scheduler that task   sample.LoadIrisData__99914b932b   has status   PENDING
[2019/09/27 16:45:24][luigi-interface][INFO](interface.py:172) Done scheduling tasks
[2019/09/27 16:45:24][luigi-interface][INFO](worker.py:1182) Running Worker with 1 processes
[2019/09/27 16:45:24][luigi-interface][INFO](worker.py:165) [pid 44268] Worker Worker(salt=954349424, workers=1, host=m3-2019mac01.local, username=shunsuke.kawai, pid=44268) running   sample.LoadIrisData()
[2019/09/27 16:45:24][gokart.file_processor][INFO](file_processor.py:73) writing a file with total_bytes=14...
[2019/09/27 16:45:24][gokart.file_processor][INFO](file_processor.py:75) writing bytes [0, 14)
[2019/09/27 16:45:24][gokart.file_processor][INFO](file_processor.py:78) done
[2019/09/27 16:45:24][gokart.file_processor][INFO](file_processor.py:73) writing a file with to

---
# Load reuslt file with thunderbolt
---

In [5]:
from thunderbolt import Thunderbolt
tb = Thunderbolt('./resource/')

100%|██████████| 1/1 [00:00<00:00, 836.69it/s]


In [6]:
tb.get_task_df()

,task_id,task_name,last_modified,task_params
0,0,LoadIrisData,2019-09-27 16:45:24.934316,{}


In [7]:
# return list
iris_df = tb.load(task_id=0)[0]

In [8]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


---
# EDA iris
---

In [9]:
import seaborn as sns
sns.set(style="ticks")

sns.pairplot(iris_df, hue='target', markers=["o", "s", "+"]).savefig('resource/iris_sample.png')

/Users/shunsuke.kawai/.pyenv/versions/3.6.8/lib/python3.6/site-packages/statsmodels/nonparametric/kde.py:487: RuntimeWarning: invalid value encountered in true_divide
  binned = fast_linbin(X, a, b, gridsize) / (delta * nobs)
/Users/shunsuke.kawai/.pyenv/versions/3.6.8/lib/python3.6/site-packages/statsmodels/nonparametric/kdetools.py:34: RuntimeWarning: invalid value encountered in double_scalars
  FAC1 = 2*(np.pi*bw/RANGE)**2


In [10]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import HTML
import japanize_matplotlib
import pandas_profiling as pdp
pdp.ProfileReport(iris_df)

Number of variables,6
Number of observations,150
Total Missing (%),0.0%
Total size in memory,7.1 KiB
Average record size in memory,48.5 B
Numeric,3
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,2


---
# Make "xgboost training and optuna tuning" gokart task with redshells
---

In [12]:
import xgboost
import redshells
from luigi import IntParameter, FloatParameter


"""-------------------------------"""
"""------ [ drop col task ] ------"""
"""-------------------------------"""

class MakeMoldelInputData(SampleTask):
    def requires(self):
        return LoadIrisData()
    
    def run(self):
        data = self.load()
        data = data.drop(['target_name'], axis=1)
        self.dump(data)


"""-----------------------------"""
"""------ [ Optuna task ] ------"""
"""-----------------------------"""

class OptimizeClassificationModel(redshells.train.train_clasification_model._ClassificationModelTask):
    task_namespace = 'sample'
    test_size = luigi.FloatParameter()  # type: float
    optuna_param_name = luigi.Parameter(description='The key of "redshells.factory.get_optuna_param".')
    output_file_path = luigi.Parameter(default='model/xgboost_model.pkl')

    def run(self):
        redshells.train.utils.optimize_model(
            self, param_name=self.optuna_param_name, test_size=self.test_size, binary=True)
        


"""------------------------------------"""
"""------ [ train XGBoost task ] ------"""
"""------------------------------------"""

class OptimizeXGBoostModel(SampleTask):
    n_estimators = IntParameter()
    test_size = FloatParameter()
    
    def requires(self):
        data = MakeMoldelInputData()
        redshells.factory.register_prediction_model('XGBClassifier', xgboost.XGBClassifier)
        return OptimizeClassificationModel(
            rerun=True,
            train_data_task=data,
            target_column_name='target',
            model_name='XGBClassifier',
            model_kwargs=dict(n_estimators=self.n_estimators),
            test_size=self.test_size,
            optuna_param_name='XGBClassifier_default')


    def run(self):
        model = self.load()
        self.dump(model)

[2019/09/27 16:45:33][tensorflow][WARNING](deprecation_wrapper.py:119) From /Users/shunsuke.kawai/.local/lib/python3.6/site-packages/redshells-0.1.7-py3.6.egg/redshells/model/early_stopping.py:19: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

[2019/09/27 16:45:33][gensim.summarization.textcleaner][INFO](textcleaner.py:37) 'pattern' package not found; tag filters are not available for English


In [13]:
# python main.py sample.OptimizeXGBoostModel --local-scheduler --no-lock
gokart.run(['sample.OptimizeXGBoostModel', '--local-scheduler',  '--no-lock', '--n-estimators=50', '--test-size=0.2'])

[2019/09/27 16:45:34][gokart.run][INFO](run.py:80) Slack notification is not activated.
[2019/09/27 16:45:34][luigi][INFO](setup_logging.py:63) logging already configured
[2019/09/27 16:45:34][luigi-interface][INFO](worker.py:595) Informed scheduler that task   sample.OptimizeXGBoostModel_50_0_2_54883dbec8   has status   PENDING
[2019/09/27 16:45:34][luigi-interface][INFO](worker.py:595) Informed scheduler that task   sample.OptimizeClassificationModel___n_estimators___XGBClassifier_XGBClassifier_de_61de70b82d   has status   PENDING
[2019/09/27 16:45:34][luigi-interface][INFO](worker.py:595) Informed scheduler that task   sample.MakeMoldelInputData__99914b932b   has status   PENDING
[2019/09/27 16:45:34][luigi-interface][INFO](worker.py:595) Informed scheduler that task   sample.LoadIrisData__99914b932b   has status   DONE
[2019/09/27 16:45:34][luigi-interface][INFO](interface.py:172) Done scheduling tasks
[2019/09/27 16:45:34][luigi-interface][INFO](worker.py:1182) Running Worker with

---
# Check optuna result parameter with thunderbolt
---

In [14]:
tb = Thunderbolt('./resource/')

df = tb.get_task_df()
df

100%|██████████| 4/4 [00:00<00:00, 2036.81it/s]


,task_id,task_name,last_modified,task_params
0,0,MakeMoldelInputData,2019-09-27 16:45:34.345099,{}
1,1,OptimizeClassificationModel,2019-09-27 16:45:41.291680,"{'train_data_task': '{""type"": ""sample.MakeMold..."
2,2,LoadIrisData,2019-09-27 16:45:24.934316,{}
3,3,OptimizeXGBoostModel,2019-09-27 16:45:41.378248,"{'n_estimators': '50', 'test_size': '0.2'}"


In [15]:
task_id = df.query('task_name=="OptimizeClassificationModel"')['task_id'].iloc[0]
print(tb.load(task_id=task_id)[0])

{'best_params': {'booster': 'gbtree', 'lambda': 2.7819594843317775e-08, 'alpha': 1.6313150348751724e-08, 'max_depth': 9, 'eta': 0.006602662985011136, 'gamma': 1.1514477793892851e-05, 'grow_policy': 'depthwise'}, 'best_value': 0.0}


---
# Train XGBoost using best param
---

In [16]:
"""------------------------------------"""
"""------ [ train XGBoost task ] ------"""
"""------------------------------------"""

from luigi.util import inherits

@inherits(OptimizeXGBoostModel)
class TrainXGBoostModel(SampleTask):
    def requires(self):
        data = MakeMoldelInputData()
        param = self.clone(OptimizeXGBoostModel).load()
        return redshells.train.TrainClassificationModel(
            rerun=True,
            train_data_task=data,
            target_column_name='target',
            model_name='XGBClassifier',
            model_kwargs=param['best_params']
        )

    def run(self):
        model = self.load()
        self.dump(model)

In [17]:
# python main.py sample.TrainXGBoostModel --local-scheduler --no-lock
gokart.run(['sample.TrainXGBoostModel', '--local-scheduler',  '--no-lock', '--n-estimators=50', '--test-size=0.2'])

[2019/09/27 16:45:42][gokart.run][INFO](run.py:80) Slack notification is not activated.
[2019/09/27 16:45:42][luigi][INFO](setup_logging.py:63) logging already configured
[2019/09/27 16:45:42][luigi-interface][INFO](worker.py:595) Informed scheduler that task   sample.TrainXGBoostModel_50_0_2_54883dbec8   has status   PENDING
[2019/09/27 16:45:42][luigi-interface][INFO](worker.py:595) Informed scheduler that task   redshells.TrainClassificationModel___booster____gbt_XGBClassifier_model_classifica_7338a43c4b   has status   PENDING
[2019/09/27 16:45:42][luigi-interface][INFO](worker.py:595) Informed scheduler that task   sample.MakeMoldelInputData__99914b932b   has status   DONE
[2019/09/27 16:45:42][luigi-interface][INFO](interface.py:172) Done scheduling tasks
[2019/09/27 16:45:42][luigi-interface][INFO](worker.py:1182) Running Worker with 1 processes
[2019/09/27 16:45:42][luigi-interface][INFO](worker.py:165) [pid 44268] Worker Worker(salt=582519396, workers=1, host=m3-2019mac01.local

---
# Check model output with thunderbolt
---

In [18]:
tb = Thunderbolt('./resource/')
df = tb.get_task_df()
df

100%|██████████| 6/6 [00:00<00:00, 7813.05it/s]


,task_id,task_name,last_modified,task_params
0,0,MakeMoldelInputData,2019-09-27 16:45:34.345099,{}
1,1,OptimizeClassificationModel,2019-09-27 16:45:41.291680,"{'train_data_task': '{""type"": ""sample.MakeMold..."
2,2,TrainClassificationModel,2019-09-27 16:45:42.829668,"{'train_data_task': '{""type"": ""sample.MakeMold..."
3,3,LoadIrisData,2019-09-27 16:45:24.934316,{}
4,4,TrainXGBoostModel,2019-09-27 16:45:42.877972,"{'n_estimators': '50', 'test_size': '0.2'}"
5,5,OptimizeXGBoostModel,2019-09-27 16:45:41.378248,"{'n_estimators': '50', 'test_size': '0.2'}"


In [19]:
model_task_id = df.query('task_name=="TrainClassificationModel"').task_id.iloc[0]
xgb_model = tb.load(model_task_id)[0]
iris_task_id = df.query('task_name=="LoadIrisData"').task_id.iloc[0]
iris_df = tb.load(iris_task_id)[0]
sample_x = iris_df.sample(1)

In [20]:
y = xgb_model.predict(sample_x[['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']].iloc[0])
print('target', sample_x['target'].iloc[0])
print('model output', y)

target 2
model output [2]


### great !!